<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#question-2" data-toc-modified-id="question-2-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>question 2</a></span></li></ul></div>

In [48]:
import numpy as np
from functools import lru_cache
from collections import Counter
import sys
import resource
sys.setrecursionlimit(1000000)
#resource.setrlimit(resource.RLIMIT_STACK, [0x1000000, resource.RLIM_INFINITY])

In [113]:
dataraw = """\
.|...\....
|.-.\.....
.....|-...
........|.
..........
.........\\
..../.\\\\..
.-.-/..|..
.|....-|.\\
..//.|....""".split("\n")
data = np.array([list(dat) for dat in dataraw])
data.shape

(10, 10)

In [137]:
with open("input.txt") as f:
    dataraw = [line.strip() for line in f]

In [138]:
data = np.array([list(dat) for dat in dataraw])
data.shape

(110, 110)

In [145]:
# \
backslash = {(1,0):(0,1),
             (0,-1):(-1,0),
             (0,1):(1,0),
             (-1,0):(0,-1)}
# /
slash = {(1,0):(0,-1),
         (0,1):(-1,0),
         (0,-1):(1,0),
         (-1,0):(0,1)}

@lru_cache(maxsize=None)
def do_step(irow, jcol, vrow, vcol):
    irow, jcol = irow + vrow, jcol + vcol

    try:
        char = data[irow, jcol]
    except IndexError:
        return
    if irow<0 or jcol<0:
        return
    
    if (irow, jcol, vrow, vcol) in seen:
        return
    else:
        seen.add((irow, jcol, vrow, vcol))

    if data[irow, jcol]=="\\":
        vrow, vcol = backslash[(vrow, vcol)]
        do_step(irow, jcol, vrow, vcol)
    elif data[irow, jcol]=="/":
        vrow, vcol = slash[(vrow, vcol)]
        do_step(irow, jcol, vrow, vcol)
    elif data[irow, jcol]=="|" and vcol:
        do_step(irow, jcol, 1, 0)
        do_step(irow, jcol,-1, 0)
    elif data[irow, jcol]=="-" and vrow:
        do_step(irow, jcol, 0, 1)
        do_step(irow, jcol, 0,-1)
    else: # if char . or v opposite to | or -
        do_step(irow, jcol, vrow, vcol)

In [146]:
irow, jcol = 0, 0
vrow, vcol = 1, 0

seen = set()
seen.add((irow, jcol, vrow, vcol))

do_step(irow, jcol, vrow, vcol)
seenX = set(s[:2] for s in seen)
len(seenX)

8539

## question 2

In [150]:
def get_n_energized(irow, jcol, vrow, vcol):
    global seen
    seen = set()
    seen.add((irow, jcol, vrow, vcol))
    do_step.cache_clear()
    do_step(irow, jcol, vrow, vcol)
    seenX = set(s[:2] for s in seen)
    return len(seenX)

In [151]:
get_n_energized(0,0,0,1)

8620

In [152]:
nmax = 0
nrow, ncol = data.shape
for i in range(nrow):
    n = get_n_energized(i, 0, 0, 1)
    if n > nmax:
        nmax = n
for i in range(nrow):
    n = get_n_energized(i, ncol, 0, -1)
    if n > nmax:
        nmax = n
for i in range(ncol):
    n = get_n_energized(0, i, 1, 0)
    if n > nmax:
        nmax = n
for i in range(ncol):
    n = get_n_energized(nrow, i, -1, 0)
    if n > nmax:
        nmax = n
nmax

8674